In [48]:
import matplotlib 
import pandas as pd
import pickle
import numpy as np
import scipy as sp
import math
import re
import pymysql as db
import os
import sklearn
import datetime
from matplotlib import pyplot as plt
from sshtunnel import SSHTunnelForwarder
from sklearn import model_selection, svm
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import classification_report, average_precision_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, roc_auc_score

from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import itertools
import time

from pyglmnet import GLM
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier

from scipy.io import loadmat
import scipy


#For Seaborn
import seaborn as sns
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,10



#For Arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

#For NLP
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer

#For Bayesian Optimization
#import bayes_opt
#from bayes_opt import BayesianOptimization


#Dask Framework
import dask.dataframe as dd

#For View
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Student/nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Student\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Student\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
#Data Ingestion
fitbit_data=pd.read_csv(r"Subject_1_Fitbit.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
wavelet_data=pd.read_csv(r"Subject_1_Wavelet.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
print(fitbit_data.shape)
print(wavelet_data.shape)

(397, 3)
(23814, 10)


# Acceleration_data

In [50]:
#sklear data ingestion
import pandas as pd

fitbit_data=pd.read_csv(r"Subject_1_Fitbit.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
wavelet_data=pd.read_csv(r"Subject_1_Wavelet.csv",sep=',',encoding="utf-8",error_bad_lines=False,low_memory = False)
print(fitbit_data.shape)
print(wavelet_data.shape)

(397, 3)
(23814, 10)


In [51]:
### Task 1 ###
fitbit_data.head()

,Time,State,Interpreted
0,05:46,2,Awake
1,05:47,2,Awake
2,05:48,2,Awake
3,05:49,2,Awake
4,05:50,2,Awake


In [23]:
wavelet_data.head()

,ts,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,timestamp_secs,timestamp_mins
0,2018-11-06T20:46:00Z,1541537160000000000,8.571167,3.165926,0.077218,NaN,NaN,NaN,05:46:00,05:46
1,2018-11-06T20:46:01Z,1541537161000000000,8.802820,3.243144,0.463306,NaN,NaN,NaN,05:46:01,05:46
2,2018-11-06T20:46:02Z,1541537162000000000,8.493949,3.243144,1.081048,NaN,NaN,NaN,05:46:02,05:46
3,2018-11-06T20:46:03Z,1541537163000000000,8.493949,3.243144,1.081048,NaN,NaN,NaN,05:46:03,05:46
4,2018-11-06T20:46:04Z,1541537164000000000,8.493949,3.243144,0.926613,NaN,NaN,NaN,05:46:04,05:46


In [52]:
### Task 2 ###
import datetime
def rev_egg(date_time):
    date_time=date_time/1000000000 #Nanoseconds to Seconds
    x=datetime.datetime.fromtimestamp(date_time).strftime('%H:%M:%S')
    return x

wavelet_data['timestamp_secs']=wavelet_data['tsUnixNano'].apply(rev_egg)

In [53]:
import datetime
def rev_egg_min(date_time):
    date_time=date_time/1000000000 #Nanoseconds to Seconds
    x=datetime.datetime.fromtimestamp(date_time).strftime('%H:%M')
    return x

wavelet_data['timestamp_mins']=wavelet_data['tsUnixNano'].apply(rev_egg_min)

In [54]:
wavelet_data.head()

,ts,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,timestamp_secs,timestamp_mins
0,2018-11-06T20:46:00Z,1541537160000000000,8.571167,3.165926,0.077218,NaN,NaN,NaN,05:46:00,05:46
1,2018-11-06T20:46:01Z,1541537161000000000,8.802820,3.243144,0.463306,NaN,NaN,NaN,05:46:01,05:46
2,2018-11-06T20:46:02Z,1541537162000000000,8.493949,3.243144,1.081048,NaN,NaN,NaN,05:46:02,05:46
3,2018-11-06T20:46:03Z,1541537163000000000,8.493949,3.243144,1.081048,NaN,NaN,NaN,05:46:03,05:46
4,2018-11-06T20:46:04Z,1541537164000000000,8.493949,3.243144,0.926613,NaN,NaN,NaN,05:46:04,05:46


In [55]:
### Task 3 ###
fitbit_start=fitbit_data['Time'][0]
fitbit_end=fitbit_data['Time'][len(fitbit_data)-1]

print(fitbit_start)
print(fitbit_end)

05:46
12:22


In [56]:
### Task 4 ###
wavelet_data['accel_x']=wavelet_data['accel_x'].apply(abs)
wavelet_data['accel_y']=wavelet_data['accel_y'].apply(abs)
wavelet_data['accel_z']=wavelet_data['accel_z'].apply(abs)

In [57]:
### Task 5 ###
agg_data=wavelet_data.groupby(['timestamp_mins']).sum()
agg_data.reset_index(inplace=True)
print(agg_data.shape)

(397, 8)


In [58]:
agg_data.head()

,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
0,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,0.0
1,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,0.0
2,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,0.0
3,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,0.0
4,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,0.0


In [59]:
### Task 6 ###
import numpy as np
import math

def resultant_vec(xaxis, yaxis, zaxis):
    xaxis=np.array(xaxis)
    yaxis=np.array(yaxis)
    zaxis=np.array(zaxis)
    resultant_vec=[]
    adder=((xaxis**2)+(yaxis**2)+(zaxis**2))
    
    for i in adder:
        resultant_vec.append(math.sqrt(i))
        
    return resultant_vec

agg_data['resultant_vector']=resultant_vec(agg_data['accel_x'], agg_data['accel_y'], agg_data['accel_z'])
agg_data

,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,resultant_vector
0,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,0.0,533.790806
1,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,0.0,528.666065
2,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,0.0,528.451256
3,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,0.0,547.716788
4,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,0.0,523.889026
...,...,...,...,...,...,...,...,...,...
392,12:18,9.249364e+19,46.330620,565.233660,275.976125,0.0,0.0,0.0,630.712643
393,12:19,9.249365e+19,46.330620,565.233660,277.983780,0.0,0.0,0.0,631.593697
394,12:20,9.249365e+19,137.293091,524.771587,254.200726,0.0,0.0,0.0,599.043087
395,12:21,9.249365e+19,314.121684,528.786939,214.896902,0.0,0.0,0.0,651.512654


In [60]:
### Task 7 ###
resultant_mean=agg_data['resultant_vector'].mean()
print(resultant_mean)

600.1735131359096


In [61]:
### Task 8 ###
def thresh_im(x):
    x=x.sort_values().unique()
    thresh=x[1]-x[0]
    return thresh

vib_thresh_x=thresh_im(agg_data['accel_x'])
vib_thresh_y=thresh_im(agg_data['accel_y'])
vib_thresh_z=thresh_im(agg_data['accel_z'])

print(vib_thresh_x)
print(vib_thresh_y)
print(vib_thresh_z)

1.23548
16.833460999999993
0.30887000000000064


In [62]:
half_window=5

def rolling_vibration_thresh(acc_list):
    zeros=np.repeat(0,half_window-1,axis=0)
    con_list=np.concatenate([zeros,acc_list])
    
    mean_list=[]
    
    for i in range(len(acc_list)):
        x=con_list[i:i+5]
        
        yolo_list=[]
        for j in range(len(x)-1):
            y=x[j+1]-x[j]
            yolo_list.append(y)
        z=np.mean(yolo_list)
        
        mean_list.append(z)
    return mean_list

In [71]:
acc_list_x=agg_data['accel_x']
acc_list_y=agg_data['accel_y']
acc_list_z=agg_data['accel_z']

vibration_x=rolling_vibration_thresh(acc_list_x)
vibration_y=rolling_vibration_thresh(acc_list_y)
vibration_z=rolling_vibration_thresh(acc_list_z)

agg_data['vibration_x']=vibration_x
agg_data['vibration_y']=vibration_y
agg_data['vibration_z']=vibration_z

In [69]:
agg_data.head(10)

,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,resultant_vector,vibration_x,vibration_y,vibration_z,x_y_corr,y_z_corr,x_z_corr
0,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,0.0,533.790806,122.872692,48.512030,18.899036,0.286848,0.348141,0.288555
1,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,0.0,528.666065,113.123958,47.450287,49.187685,0.286848,0.348141,0.288555
2,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,0.0,528.451256,64.090704,64.380270,95.923709,0.286848,0.348141,0.288555
3,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,0.0,547.716788,64.360968,51.291869,109.436808,0.286848,0.348141,0.288555
4,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,0.0,523.889026,-95.151528,3.880190,97.892763,0.286848,0.348141,0.288555
5,05:51,9.249225e+19,118.297558,225.707388,501.451861,0.0,0.0,0.0,562.487428,-83.549569,8.976560,76.175280,0.286848,0.348141,0.288555
6,05:52,9.249225e+19,417.361772,164.782610,266.401118,0.0,0.0,0.0,521.836864,40.249739,-23.184617,-29.323430,0.286848,0.348141,0.288555
7,05:53,9.249226e+19,546.701460,75.364477,61.310885,0.0,0.0,0.0,555.266887,72.314398,-32.450750,-94.109087,0.286848,0.348141,0.288555
8,05:54,9.249226e+19,546.701460,79.225368,55.596780,0.0,0.0,0.0,555.202798,108.954201,-32.585878,-102.892604,0.286848,0.348141,0.288555
9,05:55,9.249226e+19,546.701460,81.696344,55.596780,0.0,0.0,0.0,555.560781,107.100976,-36.002761,-111.463770,0.286848,0.348141,0.288555


In [72]:
# 상관관계 분석
agg_data['x_y_corr']=agg_data['accel_x'].corr(agg_data['vibration_x'])
agg_data['y_z_corr']=agg_data['accel_y'].corr(agg_data['vibration_y'])
agg_data['x_z_corr']=agg_data['accel_z'].corr(agg_data['vibration_z'])

#표본 첨도 구함
agg_data['kurt_x']=agg_data['vibration_x'].kurt()
agg_data['kurt_y']=agg_data['vibration_y'].kurt()
agg_data['kurt_z']=agg_data['vibration_z'].kurt()

#표본 비대칭도 (표본 왜도)
agg_data['skew_x']=agg_data['vibration_x'].skew()
agg_data['skew_y']=agg_data['vibration_y'].skew()
agg_data['skew_z']=agg_data['vibration_z'].skew()

#확률분포가 가지는 정보의 확신도 측정
agg_data['entropy_x']=sp.stats.entropy(agg_data['vibration_x'])
agg_data['entropy_y']=sp.stats.entropy(agg_data['vibration_y'])
agg_data['entropy_z']=sp.stats.entropy(agg_data['vibration_z'])

In [73]:
agg_data.head(10)

,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,resultant_vector,vibration_x,...,x_z_corr,kurt_x,kurt_y,kurt_z,skew_x,skew_y,skew_z,entropy_x,entropy_y,entropy_z
0,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,0.0,533.790806,122.872692,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
1,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,0.0,528.666065,113.123958,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
2,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,0.0,528.451256,64.090704,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
3,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,0.0,547.716788,64.360968,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
4,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,0.0,523.889026,-95.151528,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
5,05:51,9.249225e+19,118.297558,225.707388,501.451861,0.0,0.0,0.0,562.487428,-83.549569,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
6,05:52,9.249225e+19,417.361772,164.782610,266.401118,0.0,0.0,0.0,521.836864,40.249739,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
7,05:53,9.249226e+19,546.701460,75.364477,61.310885,0.0,0.0,0.0,555.266887,72.314398,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
8,05:54,9.249226e+19,546.701460,79.225368,55.596780,0.0,0.0,0.0,555.202798,108.954201,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
9,05:55,9.249226e+19,546.701460,81.696344,55.596780,0.0,0.0,0.0,555.560781,107.100976,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf


In [74]:
main_data=pd.merge(fitbit_data,agg_data,how='left',left_on='Time',right_on='timestamp_mins')
main_data

,Time,State,Interpreted,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,...,x_z_corr,kurt_x,kurt_y,kurt_z,skew_x,skew_y,skew_z,entropy_x,entropy_y,entropy_z
0,05:46,2,Awake,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
1,05:47,2,Awake,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
2,05:48,2,Awake,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
3,05:49,2,Awake,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
4,05:50,2,Awake,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,12:18,2,Awake,12:18,9.249364e+19,46.330620,565.233660,275.976125,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
393,12:19,2,Awake,12:19,9.249365e+19,46.330620,565.233660,277.983780,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
394,12:20,2,Awake,12:20,9.249365e+19,137.293091,524.771587,254.200726,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf
395,12:21,2,Awake,12:21,9.249365e+19,314.121684,528.786939,214.896902,0.0,0.0,...,0.288555,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf


In [76]:
main_data['Interpret_Num']=main_data['Interpreted'].apply(lambda x:0 if x=='Awake' else 1)

feature_cols=['accel_x','accel_y','accel_z','tsUnixNano',
             'resultant_vector','x_y_corr','y_z_corr','x_z_corr','kurt_x',
             'kurt_y','kurt_z','skew_x','skew_y','skew_z','entropy_x','entropy_y',
             'entropy_z']

label_cols=['Interpret_Num']

features=main_data[feature_cols]
labels=main_data[label_cols]

In [77]:
main_data

,Time,State,Interpreted,timestamp_mins,tsUnixNano,accel_x,accel_y,accel_z,gyro_x,gyro_y,...,kurt_x,kurt_y,kurt_z,skew_x,skew_y,skew_z,entropy_x,entropy_y,entropy_z,Interpret_Num
0,05:46,2,Awake,05:46,9.249223e+19,491.490768,194.048120,75.596143,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
1,05:47,2,Awake,05:47,9.249223e+19,452.495832,189.801149,196.750742,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
2,05:48,2,Awake,05:48,9.249224e+19,256.362818,257.521080,383.694838,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
3,05:49,2,Awake,05:49,9.249224e+19,257.443870,205.167476,437.747233,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
4,05:50,2,Awake,05:50,9.249225e+19,110.884656,209.568882,467.167195,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,12:18,2,Awake,12:18,9.249364e+19,46.330620,565.233660,275.976125,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
393,12:19,2,Awake,12:19,9.249365e+19,46.330620,565.233660,277.983780,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
394,12:20,2,Awake,12:20,9.249365e+19,137.293091,524.771587,254.200726,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
395,12:21,2,Awake,12:21,9.249365e+19,314.121684,528.786939,214.896902,0.0,0.0,...,5.729283,6.497155,5.795949,-0.367565,0.123749,-0.223396,-inf,-inf,-inf,0
